# Comparison between Deconvolution/Denoising and Fast Super-Resolution

## Google Colab Integration

In [591]:
# !git clone https://github.com/Stage-SuperResolution/NAME-REPO.git
# !mv NAME-REPO/* .
# !rm -rf NAME-REPO
# !pip install git+https://github.com/akhaten/lasp.git@COMPLETE-COMMIT

## Import

In [592]:
# Lasp module

import sys
sys.path.append('../..')

import lasp.io
import lasp.filters.linear
import lasp.noise
import lasp.convert
import lasp.algorithm.experimental
import lasp.thresholding
import lasp.differential
import lasp.utils

# Other

import scipy.signal
import scipy.io.matlab
import numpy
import pandas
import tqdm
import pathlib
import matplotlib.pyplot
import typing

IMAGE_PATH = pathlib.Path('./0-Images')
# PATH = pathlib.Path('./3-LaplacianVersus')
# if not(PATH.exists()):
#     PATH.mkdir()

## Test On

### Decovolution/Denoising

In [593]:
def mumford_shah_deconv_v3(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    alpha: float,
    beta: float,
    sigma: float,
    nb_iterations: int,
    tolerance: float,
    error_history: list[float] = None
) -> numpy.ndarray:

    """Mumford Shah
    # TODO: make test
    Solve argmin_{x} { (alpha/2) || y - Hx ||^2 + (beta/2) || nabla y ||^2 + || nabla y ||_1

    Difference with v2 ?
    We use Derivation in fourier space
    and we set lap_diag = Dxt * Dx + Dyt * Dy and not laplacia  filter
    """

    Dx_pad = lasp.utils.pad(numpy.array([[1, -1]]), y.shape)
    Dx = numpy.fft.fft2(Dx_pad)
    Dxt = numpy.conj(Dx)

    Dy_pad = lasp.utils.pad(numpy.transpose(numpy.array([[1, -1]])), y.shape)
    Dy = numpy.fft.fft2(Dy_pad)
    Dyt = numpy.conj(Dy)

    # Build kernel

    Dx_diag = lasp.utils.fourier_diagonalization(
        kernel = Dx_pad,
        shape_out = y.shape 
    )
    Dxt_diag = numpy.conj(Dx_diag)

    Dy_diag = lasp.utils.fourier_diagonalization(
        kernel = Dy_pad,
        shape_out = y.shape 
    )
    Dyt_diag = numpy.conj(Dy_diag)

    lap_diag = Dxt_diag * Dx_diag + Dyt_diag * Dy_diag
   
   
    h_diag = lasp.utils.fourier_diagonalization(
        kernel = h,
        shape_out = y.shape
    )

    h2_diag = numpy.abs(h_diag)**2


    uker = alpha * h2_diag + (beta+sigma) * lap_diag

    rhs1fft = alpha * numpy.conj(h_diag) * numpy.fft.fft2(y)

    # Initialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for _ in range(0, nb_iterations):

        rhs2fft = sigma*Dxt*numpy.fft.fft2(d_x-b_x)+sigma*Dyt*numpy.fft.fft2(d_y-b_y)
        rhsfft = rhs1fft + rhs2fft

        u_prev = numpy.copy(u)

        u_fft = rhsfft / uker
        u = numpy.real(numpy.fft.ifft2(u_fft))    

        err = numpy.linalg.norm(u-u_prev, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
    
        u_dx = numpy.real(numpy.fft.ifft2(Dx * u_fft))
        u_dy = numpy.real(numpy.fft.ifft2(Dy * u_fft))

        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array(
                [ 
                    u_dx + b_x, 
                    u_dy + b_y 
                ]
            ),
            epsilon = 1/sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)
    
    return u_normalized

### Fast Super-Resolution

In [594]:
def mumford_shah_fsr(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    alpha: float,
    beta0: float,
    beta1: float,
    sigma: float,
    d: int,
    nb_iterations: int,
    tolerance: float,
    gamma: float = 0.,
    error_history: list[float] = None
) -> numpy.ndarray:


    """Mumford Shah
    # TODO: make test
    Solve $$argmin_{x} { (alpha/2) || y - Hx ||^2 + (beta0/2) || nabla y ||^2 + beta1 || nabla y ||_1$$

    Params:
        - y: low resolution
        - h: deblur kernel
        - alpha: hyper parameter of data fidelity
        - beta0: hyper parameter of dirichlet energy
        - beta1: hyper parameter of total variation
        - sigma: split-bregman hyper parameter
        - d: decimation
        - nb_iterations: number of iteration
        - tolerance: tolerance
        - error_history: save errors of each iteration
    
    Returns:
        - high resolution of y
    """

    def block_mm(nr, nc, Nb, x1, order: str) -> numpy.ndarray:

        block_shape = numpy.array([nr, nc])

        x1 = lasp.utils.blockproc_reshape(x1, block_shape, order)
        x1 = numpy.reshape(x1, newshape=(nr*nc, Nb), order=order)
        x1 = numpy.sum(x1, axis=1)
        x = numpy.reshape(x1, newshape=(nr, nc), order=order)

        return x


    # print(alpha, beta0, beta1)

    y_rows, y_cols = y.shape

    Dx_pad = lasp.utils.pad(numpy.array([[1, -1]]), (d*y_rows, d*y_cols))
    Dx = numpy.fft.fft2(Dx_pad)
    Dxt = numpy.conj(Dx)

    Dy_pad = lasp.utils.pad(numpy.transpose(numpy.array([[1, -1]])), (d*y_rows, d*y_cols))
    Dy = numpy.fft.fft2(Dy_pad)
    Dyt = numpy.conj(Dy)

    # Build kernel

    Dx_diag = lasp.utils.fourier_diagonalization(
        kernel = Dx_pad,
        shape_out = (d*y_rows, d*y_cols)
    )
    Dxt_diag = numpy.conj(Dx_diag)

    Dy_diag = lasp.utils.fourier_diagonalization(
        kernel = Dy_pad,
        shape_out = (d*y_rows, d*y_cols) 
    )
    Dyt_diag = numpy.conj(Dy_diag)

    lap_diag = Dxt_diag * Dx_diag + Dyt_diag * Dy_diag  + gamma
   
    h_diag = lasp.utils.fourier_diagonalization(
        kernel = h,
        shape_out = numpy.array([d*y_rows, d*y_cols])
    )

    h_diag_transp = numpy.conj(h_diag)

    h2_diag = numpy.abs(h_diag)**2
 
    STy = numpy.zeros(shape=(d*y_rows, d*y_cols))
    STy[0::d, 0::d] = numpy.copy(y)
    rhs1fft = alpha * h_diag_transp * numpy.fft.fft2(STy)


    # Initialization
    # import PIL.Image
    # u = numpy.array(
    #     PIL.Image.Image.resize(
    #         PIL.Image.fromarray(y),
    #         (y_rows*d, y_cols*d),
    #         PIL.Image.Resampling.BICUBIC
    #     )
    # )
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(u)
    d_y=numpy.zeros_like(u)
    b_x=numpy.zeros_like(u)
    b_y=numpy.zeros_like(u)

    for _ in range(0, nb_iterations):

        rhs2fft = sigma*Dxt*numpy.fft.fft2(d_x-b_x)+sigma*Dyt*numpy.fft.fft2(d_y-b_y)
        rhsfft = rhs1fft + rhs2fft

        u_prev = numpy.copy(u)

        # Inverse
        #u_fft = rhsfft / uker
        ## Parameters
        # fr = rhsfft
        # fb = h_diag
        # fbc = numpy.conj(h_diag)
        # f2b = h2_diag
        # nr, nc = y.shape
        # m = nr * nc
        # f2d = lap_diag
        # nb = d*d
        ##
        # x1 = h_diag*rhsfft / lap_diag
        x1 = h_diag*rhsfft / lap_diag
        fbr = block_mm(y_rows, y_cols, d*d, x1, order='F')
        # invW = block_mm(y.shape[0], y.shape[1], d*d, h2_diag / lap_diag, order='F')
        invW = block_mm(y_rows, y_cols, d*d, h2_diag / lap_diag, order='F')
        # invWBR = fbr / (invW + beta1*d*d)
        invWBR = fbr / ( invW + (beta0+sigma) * (d*d / alpha) )
        fun = lambda block : block*invWBR
        FCBinvWBR = lasp.utils.blockproc(numpy.copy(h_diag_transp), numpy.array([y_rows, y_cols]), fun)
        ## Returns
        u_fft = (rhsfft - FCBinvWBR) / lap_diag
        u_fft /= (beta0 + sigma)
        # u_fft /= beta1
        ##########
        
        # u_fft = rhsfft / uker

        # Compute errors
        u = numpy.real(numpy.fft.ifft2(u_fft))    

        err = numpy.linalg.norm(u-u_prev, 'fro') / numpy.linalg.norm(u, 'fro')
        
        if not(error_history is None):
            error_history.append(err)

        if err < tolerance:
            break
        
        
        u_dx = numpy.real(numpy.fft.ifft2(Dx * u_fft))
        u_dy = numpy.real(numpy.fft.ifft2(Dy * u_fft))


        d_x, d_y = lasp.thresholding.multidimensional_soft(
            d = numpy.array(
                [ 
                    u_dx + b_x, 
                    u_dy + b_y 
                ]
            ),
            epsilon = beta1 / sigma
        )

        b_x += (u_dx - d_x)
        b_y += (u_dy - d_y)

    u_normalized = lasp.utils.normalize(u)

    return u_normalized

## Dataset utils

In [595]:
def image_from(params: pandas.Series) -> numpy.ndarray:

    image_path = params['image']
    blur = params['blur']
    noise = params['noise']
    
    img = lasp.io.read(image_path)
    out = numpy.copy(img)

    if pandas.notna(blur):
        kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1])
        out = scipy.signal.convolve2d(out, kernel, mode='same')
    
    if pandas.notna(noise):
        out = lasp.noise.awgn(out, snr=noise)

    return out

def add(
    dataset: pandas.DataFrame,
    image: pathlib.Path,
    deblur_kernel: tuple[int, float],
    alpha: float,
    beta0: float,
    beta1: float,
    sigma: float,
    d: int,
    tol: float = 10**(-4),
    iterations: int = 300,
    noise: float = numpy.nan,
    blur: tuple[int, float] = numpy.nan
) -> pandas.DataFrame:

    to_add = pandas.DataFrame(
        {
            'image' : [image], 
            'deblur_kernel' : [deblur_kernel], 
            'alpha' : [alpha], 
            'beta0' : [beta0],
            'beta1' : [beta1],
            'sigma' : [sigma],
            'd' : [d],
            'tol' : [tol], 
            'iterations' : [iterations], 
            'noise' : [noise],
            'blur' : [blur]
        }
    )

    return pandas.concat([dataset, to_add], ignore_index=True)

In [596]:
def make_dataset() -> pandas.DataFrame:    
    

    dataset = pandas.DataFrame(
        columns = [
            'image' , 'deblur_kernel',
            'alpha', 'beta0', 'beta1', 'sigma', 'd', # Hyper-parameters
            'tol', 'iterations', # Iterative parameters
            'noise',
            'blur'
        ]
    )

    dataset = add(
        dataset = dataset, 
        image = IMAGE_PATH / 'Baboon.bmp', deblur_kernel = (7, 3),
        alpha = 100, beta0 = 1, beta1 = 1, sigma = 2, d = 1,
        tol = 0, iterations = 10, 
        noise = lasp.convert.snrdb_to_snr(30),
        blur = (7, 3)
    )

    return dataset

In [597]:
# DATASET_PATH = PATH / pathlib.Path('dataset.pkl')
dataset = make_dataset()
dataset

,image,deblur_kernel,alpha,beta0,beta1,sigma,d,tol,iterations,noise,blur
0,0-Images/Baboon.bmp,"(7, 3)",100,1,1,2,1,0,10,31.622777,"(7, 3)"


## Process dataset

In [598]:
begin = 0
# dataset = pandas.read_pickle(DATASET_PATH)
dataset_filtered = dataset.loc[begin:]

res_cmp = []
# errors_cmp = []
for index in tqdm.tqdm(dataset_filtered.index):

    params = dataset.loc[index] # Get params from dataset

    input = image_from(params)
    input_normalized = lasp.utils.normalize(input)

    deblur_kernel = lasp.filters.linear.gaussian_filter(
        size = dataset.iloc[index]['deblur_kernel'][0],
        sigma = dataset.iloc[index]['deblur_kernel'][1]
    ) # Not use in function

    # Hyper-parameters
    alpha = params['alpha']
    beta0 = params['beta0']
    beta1 = params['beta1']
    sigma = params['sigma']
    d = params['d']

    # Iterative parameters
    tol = params['tol']
    iterations = params['iterations']

    res_deconv = mumford_shah_deconv_v3(
        input_normalized, deblur_kernel,
        alpha, beta0, sigma,
        iterations, tol
    )

    res_fsr = mumford_shah_fsr(
        input_normalized, deblur_kernel,
        alpha, beta0, beta1, sigma, d,
        iterations, tol, gamma=1e-16
    )
    
    res_cmp.append(numpy.max(numpy.abs(res_fsr-res_deconv)))
    # errors_cmp.append(numpy.max(numpy.abs(errors_v1-errors_v2)))

res_cmp = numpy.array(res_cmp)
# errors_cmp = numpy.array(errors_cmp)

100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


In [599]:
print(res_cmp)

[1.46882506e-12]
